In [1]:
import pandas as pd 
import numpy as np
import lxml
import traceback
import re 
from bs4 import BeautifulSoup as bs
from thefuzz import fuzz


C:\Users\Joshualevy\Anaconda3\envs\ioCapture\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
test_download = open('econ_lit_xml/res_90-21.xml', 'r', encoding='utf-8')
content = test_download.read()
test_bs_content = bs(content, 'lxml')

In [3]:
candidate_list = test_bs_content.find_all('rec')
res_list = []
print(len(candidate_list))
for candidate in candidate_list:
    try:
        journal_title = candidate.find('jtl').text
    except:
        print('failing out on record {}'.format(candidate.get('resultid')))
        continue
    if journal_title == "Review of Economic Studies":
        res_list.append(candidate)  
    else:
        print('failing out on record {}'.format(candidate.get('resultid')))

len(res_list)

1510
failing out on record 1
failing out on record 2
failing out on record 3
failing out on record 4
failing out on record 5
failing out on record 6
failing out on record 7
failing out on record 8
failing out on record 9
failing out on record 10
failing out on record 11
failing out on record 12
failing out on record 13
failing out on record 14
failing out on record 15
failing out on record 16
failing out on record 17
failing out on record 18
failing out on record 19
failing out on record 20
failing out on record 21
failing out on record 22
failing out on record 23
failing out on record 24
failing out on record 25
failing out on record 26
failing out on record 27
failing out on record 28
failing out on record 29
failing out on record 30
failing out on record 31
failing out on record 32
failing out on record 33
failing out on record 34
failing out on record 35
failing out on record 36
failing out on record 37
failing out on record 38
failing out on record 39
failing out on record 40
fail

0

In [4]:
xml_dict = {
    'doi' : 'ui',
    'title' : 'atl',
    'volume' : 'vid',
    'issue' : 'iid',
    'date' : 'dt',
    'abstract' : 'ab',
    'pages' : 'pages',
    'author' : 'au',
    'jel_desc' : 'su',
}


def field_population(temp_df, xml_obj, nav_dict, field):
    try:
        if field in ['jel_desc', 'author']:
            
            list_of = [desc.text for desc in xml_obj.find_all(nav_dict[field])]
            # print(list_of)
            temp_df.at[0,field] = list_of
            return temp_df

        value = xml_obj.find(nav_dict[field]).text
        temp_df[field] = value

        return temp_df
    except:
        # print('---------------------------------------')
        # print('fail on resultID {}, {}'.format(xml_obj.get('resultid'),field))
        value = 'ECONLIT None Found'
        temp_df[field] = value
        return temp_df
        # print(traceback.format_exc())


test_df = pd.DataFrame({})


for article in res_list:
    temp_df = pd.DataFrame({
        'doi' : [None],
        'title' : [None],
        'volume' : [None],
        'issue' : [None],
        'date' : [None],
        'abstract' : [None],
        'pages' : [None],
        'author' : [None],
        'jel_desc' : [None]
    })
    for field in xml_dict.keys():
        temp_df = field_population(temp_df, article, xml_dict, field)
    
    test_df = pd.concat([test_df, temp_df], ignore_index=True)




In [5]:
def insert_dash_date(string):
    try:
        listed = list(string)
        listed.insert(6, '-')
        listed.insert(4, '-')
        
        rejoined = ''.join(listed)
        return rejoined
    except:
        return string

test_df.date = test_df.date.apply(lambda x: insert_dash_date(x))
# test_df = test_df.explode('author')


AttributeError: 'DataFrame' object has no attribute 'date'

In [ ]:
jel_df = pd.read_xml('aea_jel_codes.xml', xpath='classification')

In [ ]:
test_merge = test_df
test_merge['id'] = np.arange(len(test_merge))
# test_merge.set_index('id', inplace=True)

test_merge = test_merge.explode('jel_desc')

test_merge = pd.merge(test_merge, jel_df, how='left', left_on='jel_desc', right_on='description')

# test_merge.index.rename("index", inplace=True)
test_merge = test_merge.groupby(by='id').agg(
    {
        'jel_desc' : lambda x: x.tolist(),
        'code' : lambda x: x.tolist(),
        'doi' : lambda x: x.unique(),
        'title' : lambda x: x.unique(),
        'volume' : lambda x: x.unique(),
        'issue' : lambda x: x.unique(),
        'date' : lambda x: x.unique(),
        'pages' : lambda x: x.unique(),
        'author' : lambda x: x,
        'abstract' : lambda x: x,
    }
).rename({'code' : 'jel_code'}, axis=1)


test_merge = test_merge.explode('doi')
test_merge = test_merge.explode('title')
test_merge = test_merge.explode('volume')
test_merge = test_merge.explode('issue')
test_merge = test_merge.explode('date')
test_merge = test_merge.explode('pages')
# test_merge = test_merge.explode('author')

for row in test_merge.index.unique().tolist(): 
    try:
        test_merge.loc[row, 'author'] = test_merge.loc[row, 'author'][0]
        if type(test_merge.loc[row, 'abstract']==list):
            test_merge.loc[row, 'abstract'] = test_merge.loc[row, 'abstract'][0]
    except:
        # print(test_merge.loc[row, 'author'])
        print(test_merge.loc[row,'title'])


test_res_codes_years = test_merge[['volume', 'issue', 'date', 'jel_desc', 'jel_code', 'title']]

Acknowledgement of Referees.
Acknowledgements.
Subscription Page.
Editorial Board.
The Quarterly Journal of Economics Volume 130 February 2015 Issue 1 - Front Cover.
Table of Content.
Subscription Page.
The Quarterly Journal of Economics Volume 129 November 2014 Issue 4 - Front Cover.
Table of Content.
Editorial Board.
Editorial Board.
Table of Content.
The Quarterly Journal of Economics Volume 129 August 2014 Issue 3 - Front Cover.
Subscription Page.
Subscription Page.
Table of Content.
Editorial Board.
The Quarterly Journal of Economics Volume 129 May 2014 Issue 2 - Front Cover.
The Quarterly Journal of Economics Volume 129 February 2014 Issue 1 - Front Cover.
Subscription Page.
Table of Content.
Editorial Board.
Editorial Board.
Table of Content.
The Quarterly Journal of Economics Volume 128 November 2013 Issue 4 - Front Cover.
Subscription Page.
Editorial Board.
Subscription Page.
Table of Content.
The Quarterly Journal of Economics Volume 128 August 2013 Issue 3 - Front Cover.
Sub

In [ ]:

test_res_codes_years['L_code'] = 0
test_res_codes_years['K_code'] = 0
test_res_codes_years['D4_code'] = 0
test_res_codes_years['G34_code'] = 0
test_res_codes_years['O3_code'] = 0

test_res_codes_years[['year','month','day']] = test_res_codes_years['date'].str.split('-', expand=True)
for row in test_res_codes_years.index.tolist():
    codes = test_res_codes_years.loc[row, 'jel_code']
    for code in codes: 
        if type(code) != str:
            continue
        if re.search(r'L', code):
            test_res_codes_years.loc[row, 'L_code'] = 1
        if re.search(r'K', code):
            test_res_codes_years.loc[row, 'K_code'] = 1
        if re.search(r'D4', code):
            test_res_codes_years.loc[row, 'D4_code'] = 1
        if re.search(r'G34', code):
            test_res_codes_years.loc[row, 'G34_code'] = 1
        if re.search(r'O3', code):
            test_res_codes_years.loc[row, 'O3_code'] = 1

test_res_codes_years

C:\Users\Joshualevy\AppData\Local\Temp\ipykernel_7776\3108175753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_qje_codes_years['L_code'] = 0
C:\Users\Joshualevy\AppData\Local\Temp\ipykernel_7776\3108175753.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_qje_codes_years['K_code'] = 0
C:\Users\Joshualevy\AppData\Local\Temp\ipykernel_7776\3108175753.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,volume,issue,date,jel_desc,jel_code,title,L_code,K_code,D4_code,G34_code,O3_code,year,month,day
id,,,,,,,,,,,,,,
0,136,1,2021-02-01,"[Consumer Economics: Theory, Asymmetric and Pr...","[D11, D82, D91]",Rational Groupthink,0,0,0,0,0,2021,02,01
1,136,1,2021-02-01,"[Firm Behavior: Empirical Analysis, Intertempo...","[D22, D25, G21, G31, G32]",The Effect of the Credit Crunch on Output Pric...,0,0,0,0,0,2021,02,01
2,136,1,2021-02-01,[Human Capital; Skills; Occupational Choice; L...,"[J24, J43, O13, O32, Q12, Q16]",Capital Obsolescence and Agricultural Producti...,0,0,0,0,1,2021,02,01
3,136,1,2021-02-01,[Asymmetric and Private Information; Mechanism...,"[D82, K42]",The Use and Misuse of Coordinated Punishments,0,1,0,0,0,2021,02,01
4,136,1,2021-02-01,"[Health Behavior, Economics of Gender; Non-lab...","[I12, J16, J46, K42, O15, O17]",Crimes against Morality: Unintended Consequenc...,0,1,0,0,0,2021,02,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1392,109,1,1994-02-01,"[Economic Growth of Open Economies, Economic I...","[F43, F15, O41]",Economic Integration and Endogenous Growth: An...,0,0,0,0,0,1994,02,01
1393,124,1,2009-02-01,[Capital Budgeting; Fixed Investment and Inven...,"[G31, G32, J16, L25, O16]",Erratum: Returns to Capital in Microenterprise...,1,0,0,0,0,2009,02,01
1394,119,4,2004-11-01,[nan],[nan],Erratum.,0,0,0,0,0,2004,11,01


In [ ]:
test_res_codes_years.groupby('year').sum().to_csv('test_res_prelim_jel_results.csv', index=False, encoding='utf-8')